In [45]:
def load_text_as_sequences(text, sequence_length=50):
    # Prétraitement du texte : suppression de sauts de ligne et des espaces superflus
    text = text.replace("\n", " ").strip()
    words = text.split()  # Diviser le texte en mots
    
    # Initialiser la liste de séquences
    sequences = []
    
    # Créer des séquences glissantes
    for i in range(len(words) - sequence_length):
        # Séquence d'entrée de longueur fixe
        input_seq = ' '.join(words[i:i + sequence_length])
        # Mot suivant à prédire
        output_word = words[i + sequence_length]
        
        # Ajouter la paire (entrée, sortie) à la liste
        sequences.append((input_seq, output_word))
    
    return sequences

# Exemple d'utilisation
text = """
Il était une fois dans une ville lointaine, un jeune garçon nommé Arthur. Arthur aimait lire des histoires de chevaliers et de dragons. Un jour, il décida de partir à l'aventure.
"""

# Extraire des paires entrée-sortie
sequences = load_text_as_sequences(text, sequence_length=10)
print(sequences[:5])  # Affiche les premières paires (entrée, sortie)


[('Il était une fois dans une ville lointaine, un jeune', 'garçon'), ('était une fois dans une ville lointaine, un jeune garçon', 'nommé'), ('une fois dans une ville lointaine, un jeune garçon nommé', 'Arthur.'), ('fois dans une ville lointaine, un jeune garçon nommé Arthur.', 'Arthur'), ('dans une ville lointaine, un jeune garçon nommé Arthur. Arthur', 'aimait')]


In [46]:
import re

def clean_moves(game_text):
    # Utilise une expression régulière pour enlever les numéros de coups (1., 2., 21., etc.)
    cleaned_text = re.sub(r"\b\d+\.\s*", "", game_text)
    return cleaned_text


def load_games_with_prefix(file_path, max_games, prefix="Début de la partie :"):
    with open(file_path, 'r') as file:
        content = file.read()

    # Diviser en parties en recherchant chaque occurrence de "[Event", indiquant le début d'une nouvelle partie
    games = content.split("[Event ")
    
    games_text = []
    i = 0
    for game in games[1:]:  # Ignorer le premier élément avant "[Event" s'il est vide
        lines = game.splitlines()
        moves = []
        result = None
        is_moves_section = False  # Indicateur pour détecter le début des coups
        
        for line in lines:
            line = line.strip()
            if line.startswith("[Result "):  # Extraire le résultat s'il est présent dans les métadonnées
                result = line.split('"')[1]
            elif not line.endswith("]") and line:  # Ignorer les lignes vides et les lignes de métadonnées
                moves.append(line)
                

        # Combiner les coups en texte unique, nettoyer et ajouter le préfixe
        if moves:  # Assurez-vous qu'il y a des coups avant d'ajouter la partie
            game_text = " ".join(moves)
            cleaned_game_text = clean_moves(game_text)  # Enlever les numéros de coups
            game_with_prefix = f"{prefix} {cleaned_game_text} {result if result else ''}".strip()
            games_text.append(game_with_prefix)
        i+=1
        if (i == max_games):
            break

    return games_text




def extract_sequences_from_text_game(game_text, max_length=120):
    # Diviser les coups et inclure le résultat final dans la séquence unique
    moves = game_text.replace("\n", " ").split(" ")
    moves = [move for move in moves if not move.isdigit()]  # Supprimer les numéros de coups

    # Extraire les séquences (entrée, sortie)
    sequences = []
    for i in range(1, len(moves) - 1):  # Ne pas inclure le dernier token dans les entrées
        input_seq = ' '.join(moves[:i])  # Tous les coups jusqu'au coup i
        output_move = moves[i]           # Coup suivant (à prédire)
        sequences.append((input_seq, output_move))

        # Limiter le nombre de séquences si nécessaire
        if len(sequences) >= max_length:
            break

    return sequences

# Chargement des parties directement en tant que texte
pgn_file_path = "/kaggle/input/pgn-files/DATA/lichess_db_standard_rated_2014-09.pgn"
games_text = load_games_with_prefix(pgn_file_path, 5000)

# Extraction des séquences pour chaque partie
all_sequences = []
i = 0
for game_text in games_text:
    i+=1
    if (i % 10000 == 0):
        print(i)
    sequences = extract_sequences_from_text_game(game_text)
    all_sequences.extend(sequences)

# Exemple de résultat
print(all_sequences[:10])  # Affiche les premières paires (entrée, sortie)


[('Début', 'de'), ('Début de', 'la'), ('Début de la', 'partie'), ('Début de la partie', ':'), ('Début de la partie :', 'e4'), ('Début de la partie : e4', 'd5'), ('Début de la partie : e4 d5', 'Nf3'), ('Début de la partie : e4 d5 Nf3', 'dxe4'), ('Début de la partie : e4 d5 Nf3 dxe4', 'Ne5'), ('Début de la partie : e4 d5 Nf3 dxe4 Ne5', 'Nf6')]


In [42]:
print(all_sequences[1])  # Affiche les premières paires (entrée, sortie)

('Début de', 'la')


In [10]:
from torch.utils.data import DataLoader, SequentialSampler
from torch.utils.data import Dataset
from transformers import GPT2Tokenizer

import random

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Utiliser le token EOS comme token de padding

class SortedBatchChessDataset(Dataset):
    def __init__(self, sequences, tokenizer, max_length=256, batch_size=4):
        # Trier les séquences par longueur
        sorted_sequences = sorted(sequences, key=lambda x: len(x[0].split()))
        
        # Organiser en blocs de `batch_size`
        self.blocks = [
            sorted_sequences[i:i + batch_size]
            for i in range(0, len(sorted_sequences), batch_size)
        ]

        # Mélanger l’ordre des blocs pour introduire de la diversité
        random.shuffle(self.blocks)

        # Aplatir la liste des blocs mélangés dans `self.sequences`
        self.sequences = [seq for block in self.blocks for seq in block]

        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        input_seq, output_move = self.sequences[idx]
        text = f"{input_seq} {output_move}"
        encodings = self.tokenizer(
            text,
            return_tensors="pt",
            max_length=self.max_length,  # Limite de longueur si nécessaire
            padding=False,               # Pas de padding ici
            truncation=True              # Tronquer si nécessaire
        )
        input_ids = encodings["input_ids"].squeeze()
        attention_mask = encodings["attention_mask"].squeeze()

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": input_ids,
        }

# Création du dataset avec bucketing
bucketed_dataset = SortedBatchChessDataset(all_sequences, tokenizer, max_length=256)

# Fonction collate_fn pour padding dynamique
def collate_fn(batch):
    max_length = max(len(item["input_ids"]) for item in batch)
    input_ids = [torch.cat([item["input_ids"], torch.full((max_length - len(item["input_ids"]),), tokenizer.pad_token_id)]) for item in batch]
    attention_mask = [torch.cat([item["attention_mask"], torch.zeros(max_length - len(item["attention_mask"]))]) for item in batch]
    input_ids = torch.stack(input_ids)
    attention_mask = torch.stack(attention_mask)
    labels = input_ids.clone()
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels,
    }

# Création du DataLoader
data_loader = DataLoader(
    bucketed_dataset,
    batch_size=4,
    collate_fn=collate_fn,
    shuffle=False
)




In [11]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW
from torch.utils.data import DataLoader
from tqdm import tqdm

# Charger le modèle et le tokenizer
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# Déplacer le modèle vers le GPU s'il est disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Configurer les paramètres d'entraînement
num_epochs = 3
learning_rate = 5e-5

# Créer l'optimiseur
optimizer = AdamW(model.parameters(), lr=learning_rate)

# Boucle d'entraînement personnalisée
model.train()
batch_loss = 0
batch_count = 0

for epoch in range(num_epochs):
    print(f"Époque {epoch + 1}/{num_epochs}")

    for i, batch in enumerate(tqdm(data_loader, desc="Batches")):
        # Charger les données du batch et les envoyer au GPU si disponible
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Accumuler la perte
        batch_loss += loss.item()
        batch_count += 1

        # Afficher la perte moyenne tous les 500 batches
        if batch_count % 500 == 0:
            avg_loss = batch_loss / 500
            print(f"Batch {batch_count}: Perte moyenne sur les 500 derniers batches: {avg_loss:.4f}")
            batch_loss = 0  # Réinitialiser la perte accumulée

print("Entraînement terminé.")



model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Époque 1/3


Batches:   0%|          | 12/97411 [00:01<3:06:33,  8.70it/s]

...

Longueur maximale : 141
Longueur minimale : 137


Batches:   0%|          | 102/97411 [00:10<2:21:05, 11.50it/s]

Longueur maximale : 88
Longueur minimale : 79


Batches:   0%|          | 109/97411 [00:11<2:44:05,  9.88it/s]


KeyboardInterrupt: 

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Charger le modèle et le tokenizer fine-tunés
model = GPT2LMHeadModel.from_pretrained("./chess_gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("./chess_gpt2")

# Fonction pour générer le prochain coup
def generate_next_move(input_sequence, max_length=20):
    # Tokeniser l'entrée
    input_ids = tokenizer.encode(input_sequence, return_tensors="pt")

    # Générer le texte (coup suivant)
    output = model.generate(
        input_ids,
        max_length=max_length,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7
    )
    
    # Décoder la sortie générée
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text

# Exemple d'utilisation
input_sequence = "1. e4 e5 2. Nf3 Nc6 3. Bb5"
predicted_move = generate_next_move(input_sequence)
print("Input Sequence:", input_sequence)
print("Predicted Move:", predicted_move[len(input_sequence):])  # Affiche le coup suivant


# OLD

In [12]:
# Fonction collate_fn pour le padding dynamique dans le DataLoader
def collate_fn(batch):
    max_length = max(len(item["input_ids"]) for item in batch)
    input_ids = [torch.cat([item["input_ids"], torch.full((max_length - len(item["input_ids"]),), tokenizer.pad_token_id)]) for item in batch]
    attention_mask = [torch.cat([item["attention_mask"], torch.zeros(max_length - len(item["attention_mask"]))]) for item in batch]
    input_ids = torch.stack(input_ids)
    attention_mask = torch.stack(attention_mask)
    labels = input_ids.clone()
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels,
    }



In [13]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Charger le tokenizer et définir le token de padding
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Utiliser le token EOS comme token de padding

class ChessDataset(torch.utils.data.Dataset):
    def __init__(self, sequences, tokenizer, max_length=256):
        self.sequences = sequences
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        input_seq, output_move = self.sequences[idx]
        
        # Ajouter l'output au bout de l'input pour le transformer en texte unique
        text = f"{input_seq} {output_move}"
        encodings = self.tokenizer(
            text, 
            return_tensors="pt", 
            max_length=self.max_length, 
            padding="max_length", 
            truncation=True
        )
        
        input_ids = encodings["input_ids"].squeeze()
        attention_mask = encodings["attention_mask"].squeeze()

        # Les labels sont identiques aux input_ids pour le modèle autoregressif GPT-2
        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": input_ids,
        }

# Créer le Dataset
dataset = ChessDataset(all_sequences, tokenizer)

In [15]:
from transformers import GPT2LMHeadModel, Trainer, TrainingArguments

# Charger le modèle
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Configurer les arguments d'entraînement
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,  # Ajuster selon les ressources de la machine
    save_steps=10_000,
    save_total_limit=2,
    logging_dir="./logs",
)

# Initialiser le Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

# Lancer le fine-tuning
trainer.train()

RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
Failed to import transformers.integrations.integration_utils because of the following error (look up to see its traceback):
Failed to import transformers.modeling_tf_utils because of the following error (look up to see its traceback):
partially initialized module 'tf_keras.src' has no attribute 'utils' (most likely due to a circular import)

## No shuffle

In [7]:
from transformers import Trainer, TrainingArguments, GPT2LMHeadModel
from torch.utils.data import DataLoader

# Charger le modèle
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Configurer les arguments d'entraînement
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,  # Ajuster selon les ressources de la machine
    save_steps=10_000,
    save_total_limit=2,
    logging_dir="./logs",
)

# Créer le DataLoader personnalisé avec collate_fn et sans shuffle
data_loader = DataLoader(dataset, batch_size=4, collate_fn=collate_fn, shuffle=False)

# Vérifier les indices
for i in range(20):
    print("Indices dans ce batch :", data_loader[i]["indices"])

# Initialiser le Trainer sans utiliser le `data_collator` mais avec le `DataLoader` personnalisé
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset  # Fournir le dataset, mais il ne sera pas utilisé pour le DataLoader
)

# Substituer le DataLoader d'entraînement par défaut avec notre DataLoader personnalisé
trainer.train_dataloader = lambda: data_loader


RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
Failed to import transformers.integrations.integration_utils because of the following error (look up to see its traceback):
Failed to import transformers.modeling_tf_utils because of the following error (look up to see its traceback):
partially initialized module 'tf_keras.src' has no attribute 'utils' (most likely due to a circular import)

In [ ]:
# Lancer le fine-tuning
trainer.train()

## Boucle d'entrainement

In [47]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW
from torch.utils.data import DataLoader
from tqdm import tqdm

# Charger le tokenizer et définir le token de padding
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Utiliser le token EOS comme token de padding

class ChessDataset(torch.utils.data.Dataset):
    def __init__(self, sequences, tokenizer, max_length=256):
        self.sequences = [(i, seq) for i, seq in enumerate(sequences)]  # Ajouter les indices
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        index, (input_seq, output_move) = self.sequences[idx]
        text = f"{input_seq} {output_move}"
        encodings = self.tokenizer(
            text, 
            return_tensors="pt", 
            max_length=self.max_length, 
            padding="max_length", 
            truncation=True
        )
        input_ids = encodings["input_ids"].squeeze()
        attention_mask = encodings["attention_mask"].squeeze()

        return {
            "index": index,  # Retourner l'indice
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": input_ids,
        }

# Créer le Dataset
dataset = ChessDataset(all_sequences, tokenizer)

In [39]:
# Fonction collate_fn pour padding dynamique
def collate_fn(batch):
    max_length = max(len(item["input_ids"]) for item in batch)
    input_ids = [torch.cat([item["input_ids"], torch.full((max_length - len(item["input_ids"]),), tokenizer.pad_token_id)]) for item in batch]
    attention_mask = [torch.cat([item["attention_mask"], torch.zeros(max_length - len(item["attention_mask"]))]) for item in batch]
    indices = [item["index"] for item in batch]
    input_ids = torch.stack(input_ids)
    attention_mask = torch.stack(attention_mask)
    labels = input_ids.clone()
    return {
        "indices": indices,  # Ajouter les indices pour vérification
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels,
    }

# Création du DataLoader
data_loader = DataLoader(
    dataset,
    batch_size=4,
    collate_fn=collate_fn,
    shuffle=True
)

In [89]:
import os
import torch
from transformers import AdamW, GPT2LMHeadModel, GPT2Tokenizer
from tqdm import tqdm

# Charger le modèle et le tokenizer
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# Déplacer le modèle vers le GPU s'il est disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Configurer les paramètres d'entraînement
num_epochs = 3
learning_rate = 5e-5
save_every = 1000  # Sauvegarder tous les 1000 batches
output_dir = "./checkpoints"  # Répertoire pour les checkpoints

# Créer le répertoire pour sauvegarder les modèles si inexistant
os.makedirs(output_dir, exist_ok=True)

# Créer l'optimiseur
optimizer = AdamW(model.parameters(), lr=learning_rate)

# Boucle d'entraînement personnalisée
model.train()
batch_loss = 0
batch_count = 0

for epoch in range(num_epochs):
    print(f"Époque {epoch + 1}/{num_epochs}")

    for i, batch in enumerate(tqdm(data_loader, desc="Batches")):
        # Charger les données du batch et les envoyer au GPU si disponible
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Accumuler la perte
        batch_loss += loss.item()
        batch_count += 1

        # Afficher la perte moyenne tous les 500 batches
        if batch_count % 500 == 0:
            avg_loss = batch_loss / 500
            print(f"Batch {batch_count}: Perte moyenne sur les 500 derniers batches: {avg_loss:.4f}")
            batch_loss = 0  # Réinitialiser la perte accumulée

        # Sauvegarder le modèle tous les 1000 batches
        if batch_count % save_every == 0:
            checkpoint_path = os.path.join(output_dir, f"checkpoint_batch_{batch_count}.pt")
            torch.save(model.state_dict(), checkpoint_path)
            print(f"Modèle sauvegardé à {checkpoint_path}")

    # Sauvegarder le modèle à la fin de chaque époque
    epoch_checkpoint_path = os.path.join(output_dir, f"checkpoint_epoch_{epoch + 1}.pt")
    torch.save(model.state_dict(), epoch_checkpoint_path)
    print(f"Modèle sauvegardé à {epoch_checkpoint_path}")

print("Entraînement terminé.")


Époque 1/3


Batches:   1%|          | 501/97411 [01:25<4:34:39,  5.88it/s]

Batch 500: Perte moyenne sur les 500 derniers batches: 0.7759


Batches:   1%|          | 517/97411 [01:28<4:35:53,  5.85it/s]


KeyboardInterrupt: 

In [88]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import StoppingCriteria, StoppingCriteriaList

# Chemin vers les fichiers sauvegardés
model_path = "./checkpoints/checkpoint_batch_1000.pt"  # Modifiez avec le chemin du fichier de checkpoint
tokenizer_path = "gpt2"  # Utilisez le tokenizer d'origine ou fine-tuné si disponible

# Charger le tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
tokenizer.pad_token = tokenizer.eos_token  # Assurez-vous que le token de padding est défini correctement

# Initialiser le modèle
model = GPT2LMHeadModel.from_pretrained("gpt2")  # Charger l'architecture de base
model.load_state_dict(torch.load(model_path))  # Charger les poids sauvegardés
model.eval()  # Mettre le modèle en mode évaluation

# Déplacer le modèle sur le GPU s'il est disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Définir un critère d'arrêt personnalisé
class StopOnSpaceCriteria(StoppingCriteria):
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def __call__(self, input_ids, scores, **kwargs):
        # Décoder les tokens générés
        decoded_text = self.tokenizer.decode(input_ids[0], skip_special_tokens=True)
        # Arrêter la génération si un espace est généré
        return " " in decoded_text[len(decoded_text.rstrip()):]

# Fonction pour générer le prochain coup
def generate_next_move(input_sequence, max_length=20):
    # Tokeniser l'entrée
    input_ids = tokenizer.encode(input_sequence, return_tensors="pt").to(device)

    stopping_criteria = StoppingCriteriaList([StopOnSpaceCriteria(tokenizer)])

    # Générer le texte (coup suivant)
    output = model.generate(
        input_ids,
        max_length=30,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
    )
    
    # Décoder la sortie générée
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text


# Exemple d'utilisation
input_sequence = "Début de la partie : e4"
predicted_move = generate_next_move(input_sequence)
print("Input Sequence:", input_sequence)
print("Predicted Move:", predicted_move[len(input_sequence):])  # Affiche le coup suivant


/tmp/ipykernel_30/4170717400.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))  # Charger les poids sauvegardés
The attentio

Input Sequence: Début de la partie : e4
Predicted Move:  c5 Nf3 d6 Bc4 Nc6 d4 exd4 e6
